In [4]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [10]:
# Download market data from Market data parsing file
market_data = pd.read_csv('market_data.csv')


In [13]:
market_data.head()

,Unnamed: 0,Date,1Y_Yield_Pct_Change_3day_avg,DXY_Pct_Change_3day_avg,10Y-2Y_Spread_Pct_Change_3day_avg,Growth-Value_Spread_Pct_Change_3day_avg
0,0,2012-01-04,0.338603,0.602934,1.534525,-1.864353
1,1,2012-01-05,-1.237343,1.061314,-0.100754,2.072520
2,2,2012-01-06,-2.505696,0.370639,-1.865863,-1.861262
3,3,2012-01-09,-1.752335,-0.233869,0.462491,3.103454
4,4,2012-01-10,1.545773,-0.308452,0.358054,1.839539


### Factor similarity regression

In [17]:
factor_similarity_df = pd.read_csv('df_min_press_speech_factor_similarity.csv')
factor_similarity_df.head()

,date,text,hawkish_similarity,dovish_similarity
0,2012-01-06,"It is the start of a new year, the traditional...",0.493903,0.446385
1,2012-01-06,\r\n Thank you for the opportunity to sp...,0.457993,0.418316
2,2012-01-07,\r\n Thank you and happy New Year. It is...,0.435671,0.370270
3,2012-01-13,\r\n It's a pleasure to be here this mor...,0.495637,0.421677
4,2012-01-16,\r\n It is certainly a pleasure to be he...,0.356629,0.327922


In [18]:
factor_similarity_df=pd.merge(factor_similarity_df, market_data, left_on='date', right_on='Date')
factor_similarity_df.head()

,date,text,hawkish_similarity,dovish_similarity,Unnamed: 0,Date,1Y_Yield_Pct_Change_3day_avg,DXY_Pct_Change_3day_avg,10Y-2Y_Spread_Pct_Change_3day_avg,Growth-Value_Spread_Pct_Change_3day_avg
0,2012-01-06,"It is the start of a new year, the traditional...",0.493903,0.446385,2,2012-01-06,-2.505696,0.370639,-1.865863,-1.861262
1,2012-01-06,\r\n Thank you for the opportunity to sp...,0.457993,0.418316,2,2012-01-06,-2.505696,0.370639,-1.865863,-1.861262
2,2012-01-13,\r\n It's a pleasure to be here this mor...,0.495637,0.421677,7,2012-01-13,-6.347302,0.817140,-4.181909,-2.086633
3,2012-01-25,"January 25, 2012 Chairman Bernanke’s Press Co...",0.418304,0.398749,14,2012-01-25,-10.765816,-0.363137,-2.809262,-6.846803
4,2012-02-10,\r\n The economic recovery began more th...,0.519238,0.386726,26,2012-02-10,-6.402795,0.700106,-3.720691,-2.320609


In [33]:
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler

target_columns = ['1Y_Yield_Pct_Change_3day_avg', 'DXY_Pct_Change_3day_avg', '10Y-2Y_Spread_Pct_Change_3day_avg', 'Growth-Value_Spread_Pct_Change_3day_avg']
dict_factor = {}

for column in target_columns:
    dict_regression = {}
    
    # Define X and Y
    X = factor_similarity_df[['hawkish_similarity', 'dovish_similarity']]
    Y = factor_similarity_df[column]
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Add a constant to the scaled features
    X_scaled_with_const = sm.add_constant(X_scaled)

    # Fit the OLS model
    result = sm.OLS(Y, X_scaled_with_const).fit()
    
    # Store results in the dictionary
    dict_regression['rsquared'] = result.rsquared
    dict_regression['hawkish__factor_tvalue'] = result.tvalues[0]
    dict_regression['dovish_factor_tvalue'] = result.tvalues[1]
    dict_factor[column] = dict_regression

# Print results to verify
print(dict_factor)

{'1Y_Yield_Pct_Change_3day_avg': {'rsquared': 0.0013180565196593275, 'hawkish__factor_tvalue': 0.7484971162792111, 'dovish_factor_tvalue': 0.9437790534079071}, 'DXY_Pct_Change_3day_avg': {'rsquared': 0.0033503723727994483, 'hawkish__factor_tvalue': -1.460407454280459, 'dovish_factor_tvalue': 1.5071877781883054}, '10Y-2Y_Spread_Pct_Change_3day_avg': {'rsquared': 0.003634987588175931, 'hawkish__factor_tvalue': 0.8062771270163834, 'dovish_factor_tvalue': 0.901079049842835}, 'Growth-Value_Spread_Pct_Change_3day_avg': {'rsquared': 0.0028969872547971454, 'hawkish__factor_tvalue': 0.9642721664714957, 'dovish_factor_tvalue': -1.5064830626916477}}


/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_37084/2794327902.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dict_regression['hawkish__factor_tvalue'] = result.tvalues[0]
/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_37084/2794327902.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dict_regression['dovish_factor_tvalue'] = result.tvalues[1]
/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_37084/2794327902.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with D

In [34]:
df_factor_results = pd.DataFrame.from_dict(dict_factor)
df_factor_results.head()

,1Y_Yield_Pct_Change_3day_avg,DXY_Pct_Change_3day_avg,10Y-2Y_Spread_Pct_Change_3day_avg,Growth-Value_Spread_Pct_Change_3day_avg
rsquared,0.001318,0.003350,0.003635,0.002897
hawkish__factor_tvalue,0.748497,-1.460407,0.806277,0.964272
dovish_factor_tvalue,0.943779,1.507188,0.901079,-1.506483


### Word list tfidf regression

In [19]:
dict_tfidf_df = pd.read_csv('dictionary_tfidf.csv')
dict_tfidf_df.head()

,Unnamed: 0,tfidf_neg,tfidf_pos,date
0,0,35.766943,8.854971,2012-01-06
1,1,18.595948,8.132138,2012-01-06
2,2,35.496525,3.690626,2012-01-07
3,3,27.907909,7.301520,2012-01-13
4,4,10.571655,8.337674,2012-01-16


In [20]:
dict_tfidf_df=pd.merge(dict_tfidf_df, market_data, left_on='date', right_on='Date')
dict_tfidf_df.head()

,Unnamed: 0_x,tfidf_neg,tfidf_pos,date,Unnamed: 0_y,Date,1Y_Yield_Pct_Change_3day_avg,DXY_Pct_Change_3day_avg,10Y-2Y_Spread_Pct_Change_3day_avg,Growth-Value_Spread_Pct_Change_3day_avg
0,0,35.766943,8.854971,2012-01-06,2,2012-01-06,-2.505696,0.370639,-1.865863,-1.861262
1,1,18.595948,8.132138,2012-01-06,2,2012-01-06,-2.505696,0.370639,-1.865863,-1.861262
2,3,27.907909,7.301520,2012-01-13,7,2012-01-13,-6.347302,0.817140,-4.181909,-2.086633
3,5,22.889941,8.760447,2012-01-25,14,2012-01-25,-10.765816,-0.363137,-2.809262,-6.846803
4,6,29.320615,4.713174,2012-02-10,26,2012-02-10,-6.402795,0.700106,-3.720691,-2.320609


In [37]:
target_columns = ['1Y_Yield_Pct_Change_3day_avg', 'DXY_Pct_Change_3day_avg', '10Y-2Y_Spread_Pct_Change_3day_avg', 'Growth-Value_Spread_Pct_Change_3day_avg']
dict_tfidf = {}

for column in target_columns:
    dict_regression = {}
    
    # Define X and Y
    X = dict_tfidf_df[['tfidf_neg', 'tfidf_pos']]
    Y = dict_tfidf_df[column]
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Add a constant to the scaled features
    X_scaled_with_const = sm.add_constant(X_scaled)

    # Fit the OLS model
    result = sm.OLS(Y, X_scaled_with_const).fit()
    
    # Store results in the dictionary
    dict_regression['rsquared'] = result.rsquared
    dict_regression['tfidf_neg_tvalue'] = result.tvalues[0]
    dict_regression['tfidf_pos_tvalue'] = result.tvalues[1]
    dict_tfidf[column] = dict_regression

# Print results to verify
print(dict_factor)

{'1Y_Yield_Pct_Change_3day_avg': {'rsquared': 0.0013180565196593275, 'hawkish__factor_tvalue': 0.7484971162792111, 'dovish_factor_tvalue': 0.9437790534079071}, 'DXY_Pct_Change_3day_avg': {'rsquared': 0.0033503723727994483, 'hawkish__factor_tvalue': -1.460407454280459, 'dovish_factor_tvalue': 1.5071877781883054}, '10Y-2Y_Spread_Pct_Change_3day_avg': {'rsquared': 0.003634987588175931, 'hawkish__factor_tvalue': 0.8062771270163834, 'dovish_factor_tvalue': 0.901079049842835}, 'Growth-Value_Spread_Pct_Change_3day_avg': {'rsquared': 0.0028969872547971454, 'hawkish__factor_tvalue': 0.9642721664714957, 'dovish_factor_tvalue': -1.5064830626916477}}


/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_37084/793029579.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dict_regression['tfidf_neg_tvalue'] = result.tvalues[0]
/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_37084/793029579.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dict_regression['tfidf_pos_tvalue'] = result.tvalues[1]
/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_37084/793029579.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame beha

In [38]:
df_tfidf_results = pd.DataFrame.from_dict(dict_tfidf)
df_tfidf_results.head()

,1Y_Yield_Pct_Change_3day_avg,DXY_Pct_Change_3day_avg,10Y-2Y_Spread_Pct_Change_3day_avg,Growth-Value_Spread_Pct_Change_3day_avg
rsquared,0.001603,0.001706,0.002242,0.000834
tfidf_neg_tvalue,0.748604,-1.459204,0.805714,0.963276
tfidf_pos_tvalue,-1.065213,0.082300,1.367815,-0.254094
